<a href="https://colab.research.google.com/github/WaiWasabi/Tensorflow-Certification-Prep/blob/main/Images/Transfer%20Learning%20and%20Fine%20Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>